In [2]:
%pip install pandas
%pip install scikit-learn

In [3]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
user_ratings_df = pd.read_csv("./data/rating.csv")
user_ratings_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [6]:
anime_metadata = pd.read_csv("./data/anime.csv")
anime_metadata = anime_metadata[['anime_id','name', 'genre']]
anime_metadata.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S..."


In [67]:
anime_data = user_ratings_df.merge(anime_metadata, on='anime_id')
anime_data.head()

,user_id,anime_id,rating,name,genre
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."
3,6,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."
4,10,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."


In [68]:

matrix = anime_data.pivot_table(
    index='anime_id', columns='user_id', values='rating', aggfunc='mean').fillna(0)
matrix

user_id,1,2,3,4,5,6,7,8,9,10,...,860,861,862,863,864,865,866,867,868,869
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,9.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,10.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Define a KNN model on cosine similarity
knn_model = NearestNeighbors(
    metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)


# Fitting the model on our matrix
knn_model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [49]:
def anime_recommender_engine(anime_id, matrix, knn_model, n_recs):
    # Fit model on matrix
    knn_model.fit(matrix)

    # Calculate neighbour distances
    distances, indices = knn_model.kneighbors(
        matrix.loc[[anime_id]], n_neighbors=n_recs)
    anime_rec_ids = sorted(list(zip(indices.squeeze().tolist(
    ), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # List to store recommendations
    cf_recs = []
    for i in anime_rec_ids:
        cf_recs.append(
            {'Name': anime_metadata['name'][i[0]], 'Distance': i[1]})

    # Select top number of recommendations needed
    df = pd.DataFrame(cf_recs, index=range(1, len(cf_recs) + 1))

    return df

In [50]:
n_recs = 10
anime_id = 20
anime_name = anime_metadata[anime_metadata['anime_id'] == 20]['name'].values[0]
print("Recommendations for the anime " + anime_name + ":")
anime_recommender_engine(
    anime_id, matrix, knn_model, n_recs)

MemoryError: Unable to allocate 6.13 GiB for an array with shape (11197, 73515) and data type float64